 # How to do nested queries

A nested query is a query within another query, meaning that you will apply a select clause on top of another select clause. Let's check the following scenario.

In [2]:
import pandas as pd
from pandas.io import gbq
# for this tutorial we will use pandas to render our data from Big Query GCP.

In an university students must subscribe to at least one sport activity. The options are Rugby (R), Basketball (B) and Jiu-Jitsu(J). After the subscription deadline we have the following lists:

**Rugby List**

In [11]:
df = gbq.read_gbq("SELECT * FROM Subcriptions2021.Rugby",project_id = "sacred-union-304216")
df

,id_subscriber,name_lastname,age,postcode
0,1,Tom Berger,25,225589
1,2,Carla Stuard,20,22245
2,10,Charls Benzon,30,22245
3,3,Jhon Merz,27,22245
4,6,Peter Gal,20,22245
5,8,Gilbert Son,29,22245
6,9,Claudia Senz,22,22264
7,12,Simon Elf,25,22264
8,4,Marlon Doislick,27,22264
9,11,Tony Sint,20,225589


**Basketball List**

In [30]:
df = gbq.read_gbq("SELECT * FROM Subcriptions2021.Basketball",project_id = "sacred-union-304216")
df

,id_subscriber,name_lastname,age,postcode
0,10,Charls Benzon,30,22245
1,16,Teresa Frank,20,22245
2,15,Diego Towers,29,22245
3,3,Jhon Merz,27,22245
4,17,Kenny Bush,27,22264
5,14,Enrique Col,32,22264
6,9,Claudia Senz,22,22264
7,13,Kiara Desma,20,22245
8,11,Tony Sint,20,225589


**Jiu-Jitsu List**

In [14]:
df = gbq.read_gbq("SELECT * FROM Subcriptions2021.JiuJitsu",project_id = "sacred-union-304216")
df

,id_subscriber,name_lastname,age,postcode
0,2,Carla Stuard,20,22245
1,6,Peter Gal,20,22245
2,20,Jaime Flowers,21,22245
3,3,Jhon Merz,27,22245
4,24,Harry Olwen,28,22245
5,22,Phill Gard,29,22245
6,10,Charls Benzon,30,22245
7,23,Thomas Muller,30,22245
8,21,Jimmy Fallo,34,22245
9,25,Violet Rock,22,22264


### First example

From the **Rugby List** we would like to know the students that are older than the average age of the Rugby's subscribers. The following query calculates the average as *age_average*

In [20]:
df = gbq.read_gbq("SELECT AVG(age) as age_average FROM Subcriptions2021.Rugby",project_id = "sacred-union-304216")
df

,age_average
0,25.875


Let's create our first nesting by filterign only students which age is *>* than *age_average*. 

In [22]:
df = gbq.read_gbq("SELECT name_lastname FROM Subcriptions2021.Rugby WHERE age > (SELECT AVG(age) as age_average FROM Subcriptions2021.Rugby)",project_id = "sacred-union-304216")
df

,name_lastname
0,Tom Berger
1,Charls Benzon
2,Jhon Merz
3,Gilbert Son
4,Simon Elf
5,Marlon Doislick
6,Marie Salck


Let's add a second nesting by filtering only the students that live in *22245*.

In [24]:
df = gbq.read_gbq("SELECT name_lastname FROM Subcriptions2021.Rugby WHERE age >(SELECT AVG(age) as age_average FROM (SELECT * FROM Subcriptions2021.Rugby WHERE postcode = '22245'))",project_id = "sacred-union-304216")
df

,name_lastname
0,Charls Benzon
1,Jhon Merz
2,Gilbert Son
3,Marlon Doislick
4,Marie Salck


### Nesting queries and joints

Nesting queries with joints is often use for solving interceptions requirements with 3 or more groups. For example, if we would like to find all students that have subscribed to the 3 sports. $R  ∩  B  ∩  J$.

Then, first let's query $R  ∩  B$, by doing a inner join and named *RugbyBasketball*:

In [31]:
df = gbq.read_gbq("SELECT Rugby.name_lastname FROM Subcriptions2021.Rugby INNER JOIN Subcriptions2021.Basketball ON Rugby.id_subscriber = Basketball.id_subscriber",project_id = "sacred-union-304216")
df

,name_lastname
0,Charls Benzon
1,Jhon Merz
2,Claudia Senz
3,Tony Sint


Now let's query $R  ∩  (B  ∩  J)$, with a new inner join on *RugbyBasketball*:

In [32]:
df = gbq.read_gbq("SELECT JiuJitsu.name_lastname FROM Subcriptions2021.JiuJitsu INNER JOIN (SELECT Rugby.id_subscriber,Rugby.name_lastname FROM Subcriptions2021.Rugby INNER JOIN Subcriptions2021.Basketball ON Rugby.id_subscriber = Basketball.id_subscriber) AS RugbyBasketball ON JiuJitsu.id_subscriber = RugbyBasketball.id_subscriber",project_id = "sacred-union-304216")
df

,name_lastname
0,Charls Benzon
1,Jhon Merz
2,Claudia Senz


### Outer Join

Now let's query the opposite result, $R △ B △ J$. In other words, all the students that are subscribed in only one sport.

First let's find the list of students that only have subscribed to Jiu-Jitsu by solving the following operation $J - (R∪B)$.

Querying $(R∪B)$:

In [62]:
df = gbq.read_gbq("SELECT * FROM (SELECT Rugby.id_subscriber, Rugby.name_lastname FROM Subcriptions2021.Rugby UNION ALL SELECT Basketball.id_subscriber, Basketball.name_lastname FROM Subcriptions2021.Basketball) as RugbyBasketball GROUP BY RugbyBasketball.id_subscriber, RugbyBasketball.name_lastname",project_id = "sacred-union-304216")
df

,id_subscriber,name_lastname
0,11,Tony Sint
1,10,Charls Benzon
2,16,Teresa Frank
3,15,Diego Towers
4,3,Jhon Merz
5,17,Kenny Bush
6,14,Enrique Col
7,9,Claudia Senz
8,13,Kiara Desma
9,2,Carla Stuard


Querying $J - (R∪B)$:

In [65]:
df = gbq.read_gbq("SELECT JiuJitsu.id_subscriber, JiuJitsu.name_lastname FROM Subcriptions2021.JiuJitsu LEFT JOIN (SELECT * FROM (SELECT Rugby.id_subscriber, Rugby.name_lastname FROM Subcriptions2021.Rugby UNION ALL SELECT Basketball.id_subscriber, Basketball.name_lastname FROM Subcriptions2021.Basketball) AS RugbyBasketball GROUP BY RugbyBasketball.id_subscriber, RugbyBasketball.name_lastname) AS RugbyBasketballUnion ON JiuJitsu.id_subscriber = RugbyBasketballUnion.id_subscriber WHERE RugbyBasketballUnion.id_subscriber IS NULL",project_id = "sacred-union-304216")
df

,id_subscriber,name_lastname
0,20,Jaime Flowers
1,24,Harry Olwen
2,22,Phill Gard
3,23,Thomas Muller
4,21,Jimmy Fallo
5,25,Violet Rock
6,18,Theodoro Stark
7,26,Sergio Tarik


 Now let's query $R - (J∪B)$, only Rugby subscription:

In [66]:
df = gbq.read_gbq("SELECT Rugby.id_subscriber, Rugby.name_lastname FROM Subcriptions2021.Rugby LEFT JOIN (SELECT * FROM (SELECT JiuJitsu.id_subscriber, JiuJitsu.name_lastname FROM Subcriptions2021.JiuJitsu UNION ALL SELECT Basketball.id_subscriber, Basketball.name_lastname FROM Subcriptions2021.Basketball) AS JiuJitsuBasketball GROUP BY JiuJitsuBasketball.id_subscriber, JiuJitsuBasketball.name_lastname) AS JiuJitsuBasketballUnion ON Rugby.id_subscriber = JiuJitsuBasketballUnion.id_subscriber WHERE JiuJitsuBasketballUnion.id_subscriber IS NULL",project_id = "sacred-union-304216")
df

,id_subscriber,name_lastname
0,8,Gilbert Son
1,4,Marlon Doislick
2,7,Selena Falc
3,5,Marie Salck


 Now let's query $B - (J∪R)$, only Basketball subscription:

In [72]:
df = gbq.read_gbq("SELECT Basketball.id_subscriber, Basketball.name_lastname FROM Subcriptions2021.Basketball LEFT JOIN (SELECT * FROM (SELECT JiuJitsu.id_subscriber, JiuJitsu.name_lastname FROM Subcriptions2021.JiuJitsu UNION ALL SELECT Rugby.id_subscriber, Rugby.name_lastname FROM Subcriptions2021.Rugby) AS JiuJitsuRugby GROUP BY JiuJitsuRugby.id_subscriber, JiuJitsuRugby.name_lastname) AS JiuJitsuRugbyUnion ON Basketball.id_subscriber = JiuJitsuRugbyUnion.id_subscriber WHERE JiuJitsuRugbyUnion.id_subscriber IS NULL",project_id = "sacred-union-304216")
df

,id_subscriber,name_lastname
0,16,Teresa Frank
1,15,Diego Towers
2,17,Kenny Bush
3,13,Kiara Desma


In [82]:
queryJ = "SELECT JiuJitsu.id_subscriber, JiuJitsu.name_lastname FROM Subcriptions2021.JiuJitsu LEFT JOIN (SELECT * FROM (SELECT Rugby.id_subscriber, Rugby.name_lastname FROM Subcriptions2021.Rugby UNION ALL SELECT Basketball.id_subscriber, Basketball.name_lastname FROM Subcriptions2021.Basketball) AS RugbyBasketball GROUP BY RugbyBasketball.id_subscriber, RugbyBasketball.name_lastname) AS RugbyBasketballUnion ON JiuJitsu.id_subscriber = RugbyBasketballUnion.id_subscriber WHERE RugbyBasketballUnion.id_subscriber IS NULL"
queryR = "SELECT Rugby.id_subscriber, Rugby.name_lastname FROM Subcriptions2021.Rugby LEFT JOIN (SELECT * FROM (SELECT JiuJitsu.id_subscriber, JiuJitsu.name_lastname FROM Subcriptions2021.JiuJitsu UNION ALL SELECT Basketball.id_subscriber, Basketball.name_lastname FROM Subcriptions2021.Basketball) AS JiuJitsuBasketball GROUP BY JiuJitsuBasketball.id_subscriber, JiuJitsuBasketball.name_lastname) AS JiuJitsuBasketballUnion ON Rugby.id_subscriber = JiuJitsuBasketballUnion.id_subscriber WHERE JiuJitsuBasketballUnion.id_subscriber IS NULL"
queryB = "SELECT Basketball.id_subscriber, Basketball.name_lastname FROM Subcriptions2021.Basketball LEFT JOIN (SELECT * FROM (SELECT JiuJitsu.id_subscriber, JiuJitsu.name_lastname FROM Subcriptions2021.JiuJitsu UNION ALL SELECT Rugby.id_subscriber, Rugby.name_lastname FROM Subcriptions2021.Rugby) AS JiuJitsuRugby GROUP BY JiuJitsuRugby.id_subscriber, JiuJitsuRugby.name_lastname) AS JiuJitsuRugbyUnion ON Basketball.id_subscriber = JiuJitsuRugbyUnion.id_subscriber WHERE JiuJitsuRugbyUnion.id_subscriber IS NULL"
querymain = "{} UNION ALL {} UNION ALL {}"
dfunion = gbq.read_gbq(querymain.format(queryJ,queryR,queryB),project_id = "sacred-union-304216")
dfunion

,id_subscriber,name_lastname
0,8,Gilbert Son
1,4,Marlon Doislick
2,7,Selena Falc
3,5,Marie Salck
4,16,Teresa Frank
5,15,Diego Towers
6,17,Kenny Bush
7,13,Kiara Desma
8,20,Jaime Flowers
9,24,Harry Olwen


Hope it was useful.